# Overcoming a Theoretical Limitation of Self-Attention 

Replication of experiments on FIRST language learning from [Overcoming a Theoretical Limitation of Self-Attention  (Chiang and Cholak, 2022)](https://arxiv.org/pdf/2202.12172.pdf).

In [1]:
import sys 
sys.path.append('../')
from src.transformer import OneTransformer
import torch

## Learning FIRST

Define training parameters as in the original paper. Citing from (David Chiang and Peter Cholak, 2020):
> We used `d_model` = 16 for word encodings, self-attention, and FFNN outputs, and `d_FFNN` = 64 for FFNN hidden layers. We used layer normalization (ε = 10^−5) after residual connections. We used PyTorch’s default initialization and trained using Adam (Kingma and Ba, 2015) with learning rate 3 × 10^−4 (Karpathy, 2016). We did not use dropout, as it did not seem to help.

In [2]:
vocab = ["0", "1", "$"]

epochs = 50
layers = 2
heads = 1 
d_model = 16
d_ffnn = 64  
eps = 1e-5 # value added to denominator in layer normalization
scaled = False

### Generalization experiment

Initialize the Transformer to learn FIRST.

In [3]:
transformer = OneTransformer(len(vocab), layers, heads, d_model, d_ffnn, scaled, eps)
optim = torch.optim.Adam(transformer.parameters(), lr=0.0003)

Define model trainer and train the transformer.

In [4]:
from src.trainer import Trainer
from src.dataset import Dataset

trainset = Dataset(0, 100, 100, random_seed=42, train=True, data_type='one', variable_lenght=False)
testset = Dataset(0, 100, 10,  random_seed=42,  train=False, data_type='one', variable_lenght=False)

trainer = Trainer(0, transformer, optim, vocab, epochs, trainset, testset, verbose=1)
train_l, val_l, train_acc, val_acc = trainer.train()

[Epoch 1] Train acc: 0.65 Train loss: 62.76117920875549, Test acc: 0.74 Test loss: 58.77211570739746
[Epoch 2] Train acc: 0.66 Train loss: 64.88557317852974, Test acc: 0.66 Test loss: 65.44172233343124
[Epoch 3] Train acc: 0.73 Train loss: 58.81425270438194, Test acc: 0.67 Test loss: 63.940348505973816
[Epoch 4] Train acc: 0.62 Train loss: 68.54289397597313, Test acc: 0.78 Test loss: 61.27927923202515
[Epoch 5] Train acc: 0.68 Train loss: 63.727036476135254, Test acc: 0.61 Test loss: 67.2500813305378
[Epoch 6] Train acc: 0.66 Train loss: 64.97652953863144, Test acc: 0.64 Test loss: 65.43881437182426
[Epoch 7] Train acc: 0.66 Train loss: 64.63837903738022, Test acc: 0.67 Test loss: 63.75513228774071
[Epoch 8] Train acc: 0.63 Train loss: 66.46105140447617, Test acc: 0.58 Test loss: 68.47598570585251
[Epoch 9] Train acc: 0.72 Train loss: 59.58387789130211, Test acc: 0.68 Test loss: 62.95991292595863
[Epoch 10] Train acc: 0.74 Train loss: 58.020698100328445, Test acc: 0.61 Test loss: 74.79

In [ ]:
import matplotlib.pyplot as plt

fig = plt.figure()
plt.plot(range(epochs), train_l, color='blue', lw=2, label="Train loss")
plt.plot(range(epochs), val_l, color='orange', lw=2, label="Validation loss")
plt.yscale('log')
ax = plt.gca()

handles, labels = ax.get_legend_handles_labels()
fig.legend(handles, labels, frameon=False, loc='lower center',  ncol=4)
plt.show()

In [ ]:
fig = plt.figure()
plt.plot(range(epochs), train_acc, color='blue', lw=2, label="Train accuracy")
plt.plot(range(epochs), val_acc, color='orange', lw=2, label="Validation accuracy")
plt.ylim([0, 1.1])

ax = plt.gca()

handles, labels = ax.get_legend_handles_labels()
fig.legend(handles, labels, frameon=False, loc='lower center',  ncol=4)
plt.show()